# Part 2： Logistic Regression with a Neural Network mindset

**原作者：Koala_Tree** [吴恩达Coursera深度学习课程 DeepLearning.ai 编程作业（1-2）](http://blog.csdn.net/koala_tree/article/details/78057033 )
 
 /* 本人在对原作者文章进行学习，文章中的部分中文解释，只是在重难点部分加深印象，稍微解释。*/

### You will learn to: 

- Build the general architecture of a learning algorithm, including: 建立学习算法的总体架构
    - Initializing parameters 初始化参数，数据
    - Calculating the cost function and its gradient  计算代价函数，还有它的 梯度\导数
    - Using an optimization algorithm (gradient descent)  使用最优化算法，梯度下降
    - Gather all three functions above into a main model function, in the right order. 将上面三个函数，使用正确的顺序整合到一个主函数中。

## 1 - Packages

First, let’s run the cell below to import all the packages that you will need during this assignment. 

- [numpy](http://www.numpy.org/) is the fundamental package for scientific computing with Python. numpy 是用来科学计算基本的库
- [h5py](http://www.h5py.org/) is a common package to interact with a dataset that is stored on an H5 file. 
- [matplotlib](https://matplotlib.org/) is a famous library to plot graphs in Python. 绘画
- [PIL](http://www.pythonware.com/products/pil/) and [scipy](https://www.scipy.org/) are used here to test your model with your own picture at the end. 最后的时候用自己的图片来测试模型。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

% matplotlib inline

## 2 - Overview of the Problem set

**Problem Statement:** You are given a dataset (“data.h5”) containing: 
- a training set of m_train images labeled as cat (y=1) or non-cat (y=0) 一个训练集，有 m 个训练数据，并有标签 cat or  non-cat
- a test set of m_test images labeled as cat or non-cat  一个测试集，images 对应的标签分类 cat or  non-cat
- each image is of shape (num_px, num_px, 3) where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).

You will build a simple image-recognition algorithm that can correctly classify pictures as cat or non-cat.
你将创建一个简单的图像识别算法，可以正确分类图片 是猫或非猫

Let’s get more familiar with the dataset. Load the data by running the following code.

In [6]:
import numpy as np
import h5py
    
    
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [9]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
print("train_set_x_orig:\n",train_set_x_orig,"\ntrain_set_y:\n",train_set_y)

train_set_x_orig:
 [[[[ 17  31  56]
   [ 22  33  59]
   [ 25  35  62]
   ..., 
   [  1  28  57]
   [  1  26  56]
   [  1  22  51]]

  [[ 25  36  62]
   [ 28  38  64]
   [ 30  40  67]
   ..., 
   [  1  27  56]
   [  1  25  55]
   [  2  21  51]]

  [[ 32  40  67]
   [ 34  42  69]
   [ 35  42  70]
   ..., 
   [  1  25  55]
   [  0  24  54]
   [  1  21  51]]

  ..., 
  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ..., 
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ..., 
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ..., 
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]]


 [[[196 192 190]
   [193 186 182]
   [188 179 174]
   ..., 
   [ 90 142 200]
   [ 90 142 201]
   [ 90 142 201]]

  [[230 229 229]
   [204 199 197]
   [193 186 181]
   ..., 
   [ 91 143 201]
   [ 91 143 201]
   [ 91 143 201]]

  [[232 225 224]
   [235 234 234]
   [208 205 202]
   ..., 
   [ 91

We added “_orig” at the end of image datasets (train and test) because we are going to preprocess 预处理 them. After preprocessing, we will end up with train_set_x and test_set_x (the labels train_set_y and test_set_y don’t need any preprocessing).

Each line of your train_set_x_orig and test_set_x_orig is an array representing an image.是一个数组，代表图像 You can visualize an example by running the following code. Feel free also to change the `index` value and re-run to see other images.

In [1]:
# Example of a picture
index = 25
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

NameError: name 'plt' is not defined